In [1]:
import pandas as pd
import numpy as np
import re
data=pd.read_csv('artifacts/Cleaned_Indian_Food_Dataset.csv',usecols=['TranslatedRecipeName','Cleaned-Ingredients','TranslatedInstructions','Ingredient-count'])

In [2]:
def clean_ingredients(ingredient_string):
    # Remove anything inside parentheses
    cleaned_string = re.sub(r'\s*\(.*?\)\s*', '', ingredient_string)
    # Convert the cleaned string to a list
    return [item.strip() for item in cleaned_string.split(',')]

# Apply the function to the 'ingredients' column and create a new column 'Ingredients cleaned'
data['Processed_Ingredients'] = data['Cleaned-Ingredients'].apply(clean_ingredients)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms.ollama import Ollama
llm = Ollama(model='mistral')
output_parser=StrOutputParser(
)
prompt3 = ChatPromptTemplate([
    ('system', """
        You are a master Indian chef specializing in traditional Indian cuisine. The user will give you a list of ingredients, and you are required to create a genuine Indian dish using only those ingredients.
        Use the following structure for your response:
        
        Ingredients Used:
            - Ingredient 1
            - Ingredient 2
            - (List all ingredients provided by the user)
        
        Instructions:
            - Step 1: (Describe each step of the cooking process in detail)
            - Step 2: (Continue outlining the steps until the dish is complete)
     
        Below are sample instructions for a dish provided. Form your sentences in a similar manner.
        - Step 1: To prepare gourd raita, prepare all the ingredients first.
        - Step 2: Add grated gourd, cucumber, curd, green chillies, salt, cumin powder and coriander in a large bowl.
        - Step 3: Mix well and your raita is ready.
        - Step 4: Serve gourd raita with Garlic Dal, gourd elder greens and phulka for dinner.
        
        Keep your response focused on 'Ingredients Used' and 'Instructions'. Avoid adding additional explanations or extra information.
    """),
    ('user', "Ingredients: {Ingredients}")
    ])
chain=prompt3|llm|output_parser

response=chain.invoke({'Ingredients':data['Processed_Ingredients'][1]})


/tmp/ipykernel_5967/4081148958.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model='mistral')


In [4]:
response

' Ingredients Used:\n       - Tomato\n       - Salt\n       - Chickpea lentils\n       - Green chilli\n       - Rice\n       - Mustard\n       - Bc belle bhat powder\n       - Dry red chilli\n       - Cashew peanuts\n       - Oil (as needed)\n       - Asafoetida (hing)\n       - Cumin seeds\n       - White urad dal\n\n   Instructions:\n       - Step 1: Soak the chickpea lentils and white urad dal for 2-3 hours.\n       - Step 2: Drain the soaked lentils and dal, then pressure cook them with salt, dry red chilli, and a little oil until soft (approx. 4-5 whistles).\n       - Step 3: In a separate pan, heat oil and add cumin seeds, followed by asafoetida.\n       - Step 4: Add the cooked lentils and dal to the pan with cumin seeds and sauté for a few minutes.\n       - Step 5: Roast the cashew peanuts in a separate pan until golden brown.\n       - Step 6: Cook the rice according to package instructions, then fluff it using a fork.\n       - Step 7: Sprinkle mustard and bc belle bhat powd

In [5]:
import re

def parse_response(response: str):
    lines = response.split('\n')
    gen_ingredients = []
    gen_instructions = []
    in_ingredients = False
    in_instructions = False
    
    for line in lines:
        line = line.strip() 
        if line == "Ingredients Used:":
            in_ingredients = True
            continue
        elif line == "Instructions:":
            in_ingredients = False
            in_instructions = True
            continue
        
        if in_ingredients and line.startswith("*"):
            # Extract only the ingredient name (before any parentheses)
            ingredient = line[1:].strip().split('(')[0].strip()
            gen_ingredients.append(ingredient)
        elif in_instructions:
            # Remove any leading numbers and periods
            instruction = re.sub(r'^\d+\.\s*', '', line)
            if instruction:  # Only add non-empty instructions
                gen_instructions.append(instruction.strip())

    return gen_ingredients, gen_instructions




gen_ingredients, gen_instructions = parse_response(response)
print("Ingredients:", gen_ingredients)
print("Instructions:", gen_instructions[1:])


Ingredients: []
Instructions: ['- Step 2: Drain the soaked lentils and dal, then pressure cook them with salt, dry red chilli, and a little oil until soft (approx. 4-5 whistles).', '- Step 3: In a separate pan, heat oil and add cumin seeds, followed by asafoetida.', '- Step 4: Add the cooked lentils and dal to the pan with cumin seeds and sauté for a few minutes.', '- Step 5: Roast the cashew peanuts in a separate pan until golden brown.', '- Step 6: Cook the rice according to package instructions, then fluff it using a fork.', '- Step 7: Sprinkle mustard and bc belle bhat powder over the cooked rice.', '- Step 8: Garnish the cooked lentils with roasted cashew peanuts and serve it with the flavored rice for dinner.']


Ingredients: ['Tomato', 'Salt', 'Chickpea lentils', 'Green chilli', 'Rice', 'Mustard', 'Bc belle bhat powder', 'Dry red chilli', 'Cashew peanuts', 'Oil (as needed)', 'Asafoetida (hing)', 'Cumin seeds', 'White urad dal']
Instructions: ['Step 1: Soak the chickpea lentils and white urad dal for 2-3 hours.', 'Step 2: Drain the soaked lentils and dal, then pressure cook them with salt, dry red chilli, and a little oil until soft (approx. 4-5 whistles).', 'Step 3: In a separate pan, heat oil and add cumin seeds, followed by asafoetida.', 'Step 4: Add the cooked lentils and dal to the pan with cumin seeds and sauté for a few minutes.', 'Step 5: Roast the cashew peanuts in a separate pan until golden brown.', 'Step 6: Cook the rice according to package instructions, then fluff it using a fork.', 'Step 7: Sprinkle mustard and bc belle bhat powder over the cooked rice.', 'Step 8: Garnish the cooked lentils with roasted cashew peanuts and serve it with the flavored rice for dinner.']


In [71]:
actual_ingredients=data['Processed_Ingredients'][1]
gen_ingredients_lower = [ingredient.lower() for ingredient in gen_ingredients]

total_input_ingredients = len(actual_ingredients)
total_ingredients_used = len(gen_ingredients_lower)

relevant_ingredients_used = len(set(actual_ingredients).intersection(set(gen_ingredients_lower)))

precision = relevant_ingredients_used / total_ingredients_used
recall = relevant_ingredients_used / total_input_ingredients

print(precision)
print(recall)

0.8333333333333334
0.8333333333333334


In [73]:
gen_instructions_cleaned='\n '.join(gen_instructions)
gen_instructions_cleaned
from nltk.translate.bleu_score import sentence_bleu


# Tokenize the sentences
reference_tokens = data['TranslatedInstructions'][0].split('/n')  # Reference needs to be a list of token lists
candidate_tokens = gen_instructions_cleaned
weights = (0.5,0.5)
# Calculate BLEU score
bleu_score = sentence_bleu(reference_tokens, candidate_tokens,weights=weights)
bleu_score

0.5327862467147153

In [10]:
#Mistral

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms.ollama import Ollama
llm = Ollama(model='mistral')
output_parser=StrOutputParser(
)
prompt2 = ChatPromptTemplate([
    ('system', """
        You are a highly skilled Indian chef. The user will provide a list of ingredients, and your responsibility is to craft an authentic Indian dish using only those ingredients. 
        Please format your response using the following structure:
        
        Ingredients Used:
            - Ingredient 1
            - Ingredient 2
            - (Include all provided ingredients)
        
        Instructions:
            - Step 1: (Provide precise cooking instructions in a step-by-step manner)
            - Step 2: (Detail each step until the recipe is complete)
     
        Below are sample instructions for a dish provided. Form your sentences in a similar manner.
        - Step 1: To prepare gourd raita, prepare all the ingredients first.
        - Step 2: Add grated gourd, cucumber, curd, green chillies, salt, cumin powder and coriander in a large bowl.
        - Step 3: Mix well and your raita is ready.
        - Step 4: Serve gourd raita with Garlic Dal, gourd elder greens and phulka for dinner.
        
        Ensure your response only includes the 'Ingredients Used' and 'Instructions'. Avoid unnecessary commentary or extra information.
    """),
    ('user', "Ingredients: {Ingredients}")
    ])
chain=prompt2|llm|output_parser

response=chain.invoke({'Ingredients':data['Processed_Ingredients'][1]})

print(response)

# Extracting ingredients
ingredients_section = response.split("Ingredients Used:")[1].split("Instructions:")[0].strip()
gen_ingredients = [ingredient.strip('- ').strip() for ingredient in ingredients_section.split('\n') if ingredient]

# Extracting instructions
instructions_section = response.split("Instructions:")[1].strip()
gen_instructions = [instruction.strip('- ').strip() for instruction in instructions_section.split('\n') if instruction]

print("Ingredients:", gen_ingredients)
print("Instructions:", gen_instructions)



actual_ingredients=data['Processed_Ingredients'][1]
gen_ingredients_lower = [ingredient.lower() for ingredient in gen_ingredients]

total_input_ingredients = len(actual_ingredients)
total_ingredients_used = len(gen_ingredients_lower)

relevant_ingredients_used = len(set(actual_ingredients).intersection(set(gen_ingredients_lower)))

precision = relevant_ingredients_used / total_ingredients_used
recall = relevant_ingredients_used / total_input_ingredients

print(precision)
print(recall)


gen_instructions_cleaned='\n '.join(gen_instructions)
gen_instructions_cleaned
from nltk.translate.bleu_score import sentence_bleu


# Tokenize the sentences
reference_tokens = data['TranslatedInstructions'][0].split('/n')  # Reference needs to be a list of token lists
candidate_tokens = gen_instructions_cleaned
weights = (0.5,0.5)
# Calculate BLEU score
bleu_score = sentence_bleu(reference_tokens, candidate_tokens,weights=weights)
bleu_score

 Ingredients Used:
        - Tomato
        - Salt
        - Chickpea lentils (Chana Dal)
        - Green chilli
        - Rice
        - Mustard (Rai)
        - BC Belle Bhat Powder (optional)
        - Dry red chilli (whole)
        - Cashew peanuts (mix of cashews and peanuts)
        - Oil (Tail Tali/Oil)
        - Asafoetida (Hing)
        - Cumin seeds (Jeera)
        - White urad dal (Urad Dal)

   Instructions:
       - Step 1: Cook rice separately and keep it aside. Soak Chickpea lentils for about 30 minutes, then pressure cook with enough water until tender.
       - Step 2: Heat oil in a pan, add cumin seeds and let them splutter. Add dry red chilli, asafoetida, and sauté the spices for a few seconds.
       - Step 3: Add chopped tomatoes to the pan and cook until they become soft. Then, add the cooked chickpea lentils along with the cooking liquid (adjust consistency according to preference).
       - Step 4: Simmer the mixture on low heat, stirring occasionally, for about 

0.7332057422104357